In [ ]:
import tensorflow as tf
import numpy as np
import os
import time


In [ ]:

test='/content/drive/My Drive/test.txt'
text1 = open(test,'rb').read().decode(encoding='cp1251')
text=text1.replace("<eos>","\n")
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1944400 characters


In [ ]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

62 unique characters


In [ ]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '$' :   4,
  '%' :   5,
  "'" :   6,
  '(' :   7,
  ')' :   8,
  '*' :   9,
  ',' :  10,
  '.' :  11,
  '0' :  12,
  '1' :  13,
  '2' :  14,
  '3' :  15,
  '4' :  16,
  '5' :  17,
  '6' :  18,
  '7' :  19,
  ...
}


In [ ]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

w
i
t
h
 


In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'with clytia he no longer was received \n than while he was a man of wealth believed \n balls , concerts'
" , op'ras , tournaments , and plays \n expensive dresses , all engaging ways \n were used to captivate "
'this lady fair \n while scarcely one around but in despair \n wife , widow , maid , his fond affection '
"sought \n to gain him , ev'ry wily art was brought \n but all in vain :пїЅ by passion overpow'red \n the"
' belle , whose conduct others would have soured \n to him appeared a goddess full of charms \n superior'


In [ ]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'with clytia he no longer was received \n than while he was a man of wealth believed \n balls , concert'
Target data: 'ith clytia he no longer was received \n than while he was a man of wealth believed \n balls , concerts'


In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("Step {:4d}".format(i))
  print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 53 ('w')
  expected output: 39 ('i')
Step    1
  input: 39 ('i')
  expected output: 50 ('t')
Step    2
  input: 50 ('t')
  expected output: 38 ('h')
Step    3
  input: 38 ('h')
  expected output: 1 (' ')
Step    4
  input: 1 (' ')
  expected output: 33 ('c')


In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embedding_dim = 256
# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape)

(64, 100, 62)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           15872     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 62)            63550     
Total params: 4,017,726
Trainable params: 4,017,726
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
sampled_indices

array([ 5, 53,  3,  6, 32, 16, 10, 24,  9,  0,  9, 27, 57, 51, 38, 11, 59,
       16,  0, 16, 13, 34, 43, 56, 27, 50,  4, 37, 18, 11, 61, 49, 30, 23,
       35, 33, 17, 42, 11, 41, 54, 53,  5, 14, 46, 49, 30, 15, 47, 27, 55,
        8, 18, 38, 44,  5, 19, 23, 27, 23, 27, 18,  0, 14, 14,  6, 12, 19,
       45,  4, 19, 50, 60, 23, 55, 57, 42, 54,  0,  3, 34, 39, 34, 38, 15,
       57, 19, 11,  0, 42, 41, 26, 51, 44, 17, 20, 41, 53,  8, 58])

In [ ]:
print("Input:", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: " 'd , nor won by love \n ambition was her crime ; but meaner folly \n dooms me to loathe , at once , a"

Next Char Predictions: 
 '%w"\'b4,<*\n*@{uh.Ѕ4\n41dmz@t$g6.їs`;ec5l.kxw%2ps`3q@y)6hn%7;@;@6\n22\'07o$7tп;y{lx\n"didh3{7.\nlk?un58kw)}'


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape)
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 62)
scalar_loss:       4.1258225


In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(dataset, epochs=3, callbacks=[checkpoint_callback])

Epoch 1/3
300/300 [==============================] - 14s 46ms/step - loss: 2.2333
Epoch 2/3
300/300 [==============================] - 14s 46ms/step - loss: 1.6751
Epoch 3/3
300/300 [==============================] - 14s 46ms/step - loss: 1.5127


In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_3'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            15872     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 62)             63550     
Total params: 4,017,726
Trainable params: 4,017,726
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 630
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temperature = 1.2

  model.reset_states()
  for i in range(num_generate):
    # using a categorical distribution to predict the character returned by the model
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)

    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
slash=input("Enter the starting string:")
slash=slash.lower()
print("\n The poem is:\n")
print(generate_text(model, start_string=slash))

Enter the starting string:hai

 The poem is:

haint , and clands words fiam keep it thyself to sound how thoughts heaves with grace as that men 
 well i sea and dabgling father blows on пїЅ oceaning phanis aribet 
 round to boend the rivh op was thing 's greenige 
 like paints most rold 's shepry , follow mure 
 and fight his elter offend oke to goiling aisse 
 twice heart drops mother dray hee for me upon the monemon prins that does 
 my love woman dead mades upon that sleep 'd is noty ) пїЅ ex , as sicky ve grows are incladed 
 beeat havusely nono turn 
 her speased than her scagels of tasencess blodded and rew 
 and mighty sted cropn the daysoe sace , winedlas aps od 


In [ ]:
#secondary API interfacing
#code to access telegram 

import json 
import requests
#here token refers to your bot API token
TOKEN = "1225137884:AAH_8CYyi4RP_tiNf9pKSpbZjy9qAAJRqzE"
URL = "https://api.telegram.org/bot{}/".format(TOKEN)


def get_url(url):
    response = requests.get(url)
    content = response.content.decode("utf8")
    return content


def get_json_from_url(url):
    content = get_url(url)
    js = json.loads(content)
    return js


def get_updates():
    url = URL + "getUpdates"
    js = get_json_from_url(url)
    return js


def get_last_chat_id_and_text(updates):
    num_updates = len(updates["result"])
    last_update = num_updates - 1
    text = updates["result"][last_update]["message"]["text"]
    chat_id = updates["result"][last_update]["message"]["chat"]["id"]
    return (text, chat_id)


def send_message(text, chat_id):
    url = URL + "sendMessage?text={}&chat_id={}".format(text, chat_id)
    get_url(url)
   


In [ ]:
text,chat=get_last_chat_id_and_text(get_updates())
(text,'r')
text.replace('/','')
text=text.lower()
print('the poem for the given string is:\n')
out=generate_text(model,start_string=text)
print(out)
send_message(out,chat)

the poem for the given string is:

awesome black 
 someword would weyth , there wonter , spredded enterth an easerol lamos in equ'st 
 what 's hot equal zenuous and my sireless prime 
 with neither пїЅ never by ravingus ( as him shed beneath to see 
 recants shoutm wave to love through thee lofty dants 

if fruitopsma that of from charted hid his heart 
 ye marlit fillet ;пїЅ alace me love sung i fed is no arguaf пїЅ stifffixned 
 ad mock hud disdradres soldret my ratherate 
 sie , when he doast doomed 
 anotherm , cold of your never down 

shoked the dears led heart done hands 

his eeeter 's briden all unto trmass ! lupes a mesi man reas 'd 
 but that she longes
